In [ ]:
import keras
import tensorflow as tf
from keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras.losses import binary_crossentropy
import os
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from scipy import stats
import matplotlib.pyplot as plt
import time
from keras.models import Sequential
import copy
import random
from operator import attrgetter
from tensorflow.keras import initializers



##############################################################################################################
# Evolutionary Settings

POPULATION_SIZE = 30
MUTATION_RATE = 0.30
GENERATIONS = 10
ELITISM = 0.80
NUMBER_CHILDS = 8 


##############################################################################################################
# Identifiability restriction

def a_reg(a):
    b = a * K.cast(K.greater_equal(a, 0), K.floatx())
    c = b + K.cast(K.equal(b, 0), K.floatx())/10
    return K.exp(K.log(c)-1/nit*K.sum(K.log(c)))
 
def b_reg(b):
    return b-K.sum(b)/nit;   


def sampling(args):
    z_mean, z_log_var, n_latent = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], n_latent))
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon



##############################################################################################################
# Autoencoder Estimation

def AEIRT(n_layers, vector_neurons, n_items, data, epochs, batch, restriction, n_latent = 1):

    inputs = Input(shape=(n_items,))
    
    if restriction == 'VAE':
        if(n_layers == 0):
            z_mean = Dense(n_latent, activation = 'linear')(inputs)
            z_log_var = Dense(n_latent, activation = 'linear')(inputs)
        else:
            x1 = Dense(vector_neurons[0], activation = 'relu')(inputs)
            for i in range(n_layers-1):
                x1 = Dense(vector_neurons[i+1], activation = 'relu')(x1)
            z_mean = Dense(n_latent, activation = 'linear')(x1)
            z_log_var = Dense(n_latent, activation = 'linear')(x1)    

        z = Lambda(sampling)([z_mean, z_log_var, n_latent])
    
        decoder_inputs = Input(shape=(n_latent,))

        outputs = Dense(n_items, 
                        activation='sigmoid',
#                        kernel_regularizer=tf.keras.regularizers.L1(0.01),
                        kernel_initializer=initializers.Ones(),
                        bias_initializer=initializers.Zeros())(decoder_inputs)
        encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
        decoder = Model(decoder_inputs, outputs, name='decoder')

        outputs = decoder(z)
        vae = Model(inputs, outputs, name='vae')

    
        reconstruction_loss = binary_crossentropy(inputs, outputs)
        reconstruction_loss *= n_items
        kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
        kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
        vae.add_loss(vae_loss)
    
        vae.compile(optimizer='adam')
    
        history = vae.fit(data,
                            epochs=epochs,
                            shuffle = True, 
                            batch_size=batch,
                            validation_split = 0.2,
                            verbose = 0,callbacks=[callback])
    
        latent = encoder.predict(data, verbose = 0)[0].flatten()
    
    else: 
        if(n_layers == 0):
            encoded = Dense(units=1, activation='linear')(inputs)
        else:
            x1 = Dense(vector_neurons[0], activation = 'relu')(inputs)
            for i in range(n_layers-1):
                x1 = Dense(vector_neurons[i+1], activation = 'relu')(x1)
            encoded = Dense(units=1, activation='linear')(x1)    
    
        decoder_inputs = Input(shape=(1,))
    
        # Op 1
        if restriction == 'Product':
            outputs = Dense(n_items, activation='sigmoid', 
                        bias_constraint=b_reg,
                        kernel_constraint=a_reg,
                        kernel_initializer=initializers.Ones(),
                        bias_initializer=initializers.Zeros())(decoder_inputs)
            encoder = Model(inputs, encoded, name='encoder')
            decoder = Model(decoder_inputs, outputs, name='decoder')

        elif restriction == 'Last Item':
            outputs = Dense(n_items-1, activation='sigmoid',
                        kernel_initializer=initializers.Ones(),
                        bias_initializer=initializers.Zeros())(decoder_inputs)
            outputs2 = Dense(units=1, activation='sigmoid',
                       kernel_initializer= keras.initializers.Ones(),
                       bias_initializer = keras.initializers.Zeros(),
                       trainable = False)(decoder_inputs)
    
            combined_input = keras.layers.concatenate([outputs, outputs2])

            encoder = Model(inputs, encoded, name='encoder')
            decoder = Model(decoder_inputs, combined_input , name='decoder')
    
        else:
            print('Restriction not found')
            return None, None, None

        outputs = decoder(encoded)
        vae = Model(inputs, outputs, name='vae')

        vae.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    
        history = vae.fit(data,data,
                    epochs=epochs,
                    shuffle = True, 
                    batch_size=batch,
                    validation_split = 0.2,
                    verbose = 0,callbacks=[callback])
    
        latent = encoder.predict(data, verbose = 0).flatten()
    
    return history, decoder, latent


##############################################################################################################
# Evolutionary algorithm


class Individual:
    def __init__(self, n_layers, vector_neurons):
        self.n_layers = n_layers
        self.vector_neurons = vector_neurons
        self.fitness = None
        self.discr = None
        self.diff = None
        self.latent = None
    
    def evaluate_fitness(self, data, epochs, batch, restriction):               
        history, decoder, latent = AEIRT(self.n_layers, self.vector_neurons, nit, data, epochs, batch, restriction)
        self.fitness = history.history['val_loss'][-1]
        self.discr = decoder.get_weights()[0]
        self.diff = decoder.get_weights()[1]
        self.latent = latent
        
def generate_individual():
    n_layers = random.randint(0, 5)
    vector_neurons = []
    for i in range(n_layers):
        aux = random.randint(2, nit*2)
        aux = aux // 5 * 5 
        aux += 1
        vector_neurons.append(aux)
    return Individual(n_layers, vector_neurons)

def crossover(parent1, parent2):
      
    child1 = Individual(parent1.n_layers, parent1.vector_neurons)
    child2 = Individual(parent2.n_layers, parent2.vector_neurons)
        
# It is randomized the changes in the vector of neurons
    for i in range(child1.n_layers):
        if parent1.n_layers <= i:
            child1.vector_neurons[i] = parent2.vector_neurons[i]
        elif parent2.n_layers <= i:
            child1.vector_neurons[i] = parent1.vector_neurons[i]
        else:
            if random.random() < 0.5:  
                child1.vector_neurons[i] = parent2.vector_neurons[i]
            else:
                child1.vector_neurons[i] = parent1.vector_neurons[i]
                
    for i in range(child2.n_layers):
        if parent1.n_layers <= i:
            child2.vector_neurons[i] = parent2.vector_neurons[i]
        elif parent2.n_layers <= i:
            child2.vector_neurons[i] = parent1.vector_neurons[i]
        else:
            if random.random() < 0.5:  
                child2.vector_neurons[i] = parent2.vector_neurons[i]
            else:
                child2.vector_neurons[i] = parent1.vector_neurons[i]
                
    return child1, child2

def mutate(individual):
    if random.random() < MUTATION_RATE:
        individual.n_layers += random.randint(-1, 1)
        if individual.n_layers < 0:
            individual.n_layers = 0
        if individual.n_layers > 5:
            individual.n_layers = 5
                
        if individual.n_layers > len(individual.vector_neurons):
            aux = random.randint(2, nit*2)
            aux = aux // 5 * 5 
            individual.vector_neurons.append(aux)
        if individual.n_layers < len(individual.vector_neurons):
            del individual.vector_neurons[-1]
                
    for i in range(len(individual.vector_neurons)):
        if random.random() < 0.5:
            aux = random.randint(-10, 10)
            aux = aux // 5 * 5
            individual.vector_neurons[i] += aux
            if individual.vector_neurons[i] < 1:
                individual.vector_neurons[i] = 2
    return individual

def select_parents(population, k):
    tournament = random.sample(population, k)
    parent1 = max(tournament, key=lambda x: x.fitness)
    tournament.remove(parent1)
    parent2 = max(tournament, key=lambda x: x.fitness)  
    return parent1, parent2

def evolve(population, data, epochs, batch, restriction):
    for i in range(GENERATIONS):
        print(f"Generation {i+1}")
        population.sort(key=attrgetter('fitness'))
        elite_count = int(ELITISM * len(population))
        new_population = population[:elite_count]
        for j in range(NUMBER_CHILDS):
            if random.random() < 0.6:
                parent1, parent2 = select_parents(population,6)
                parent1 = copy.deepcopy(parent1)
                parent2 = copy.deepcopy(parent2)  
                child1, child2 = crossover(parent1, parent2)
            else:
                child1 = generate_individual()
                child2 = generate_individual()
            child1 = mutate(child1)
            child2 = mutate(child2)

            try:
                child1.evaluate_fitness(data, epochs, batch, restriction)
            except:
                child1.fitness = 100


            try:
                child2.evaluate_fitness(data, epochs, batch, restriction)
            except:
                child2.fitness = 100
                
            new_population.append(child1)
            new_population.append(child2)
        population = new_population
        population.sort(key=attrgetter('fitness'))
        population = population[:POPULATION_SIZE]     

#     Only if we want to check how the population is evolving
#        for i in range(POPULATION_SIZE):
#            print(f"Layers: {population[i].n_layers} -- {population[i].vector_neurons}")

    population.sort(key=attrgetter('fitness'))
    return population


# Product

# VAE

# Last Item

In [ ]:
# Evolutionary process with the simulated data for simulation 1

nit = 50
nEx = 1000
Npop = 1

epochs=10000
batch=64

restrictionU = 'Last Item'

random.seed(10)

tf.config.experimental.enable_op_determinism()
keras.utils.set_random_seed(50)



callback = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-8,patience=50,
                                         verbose=0,mode="min",
                                        restore_best_weights=True)


data = pd.read_csv("DataEvo/Data_1.csv")

##############################################################################################################
# Network  
        
individuals = []
for j in range(POPULATION_SIZE):
    individual = generate_individual()
    try:
        individual.evaluate_fitness(data.to_numpy(), epochs, batch, restrictionU)
    except:
        individual.fitness = 100
    print(individual.fitness)
    individuals.append(individual)
        
pops = evolve(population = individuals,data=data.to_numpy(), epochs=epochs, batch=batch, restriction = restrictionU)



In [ ]:
# Returning the estimated parameters for the best-estimated networks


parameter = pd.DataFrame()
for i in range(POPULATION_SIZE):
    column_name = f'column_{i}'  
    parameter[column_name] = pops[i].discr.flatten() 

parameter.to_csv('Outs/aLIIC.csv')

parameter = pd.DataFrame()
for i in range(POPULATION_SIZE):
    column_name = f'column_{i}'  
    parameter[column_name] = pops[i].diff.flatten() 

parameter.to_csv('Outs/bLIIC.csv')    
    

parameter = pd.DataFrame()
for i in range(POPULATION_SIZE):
    column_name = f'column_{i}'  
    parameter[column_name] = pops[i].latent.flatten() 

parameter.to_csv('Outs/thetasLIIC.csv')     



    

In [ ]:
fitness_list = []

for i in range(POPULATION_SIZE):
    fitness_list.append(pops[i].fitness)


fitness_list = pd.DataFrame(fitness_list)


In [ ]:
fitness_list.to_csv('Outs/LIfitness.csv')

In [ ]:
for i in range(POPULATION_SIZE):
    print(pops[i].vector_neurons)

In [ ]:
for i in range(POPULATION_SIZE):
    print(pops[i].fitness)